In [ ]:
import tensorflow as tf
import data.imageReading as ir
from fairness import fairnessMetrics as fm
from model import model as m
from model import utils as utils
from model import biasMitigation as mit
from model import evaluation as ev
import datetime
import os

In [ ]:
preprocess_input = tf.keras.applications.vgg16.preprocess_input

image_size = (224,224)
batch_size = 8
epochs = 60
(ds_train, train_batches, ds_val, val_batches, ds_test, test_batches, count_classes) = ir.readData("/mimer/NOBACKUP/groups/snic2022-22-1091/museumFaces", image_size, batch_size, preprocess_input)

class_weight = mit.findClassWeights(train_batches)

strategy = tf.distribute.MirroredStrategy()

with strategy.scope():
    base_model = m.build_model()
    metric_list = m.metrics_list()

base_model.compile(optimizer=tf.keras.optimizers.Adam(learning_rate=1e-3), 
                           loss="binary_crossentropy", 
                           metrics=metric_list)

options = tf.data.Options()
options.experimental_distribute.auto_shard_policy = tf.data.experimental.AutoShardPolicy.OFF
ds_train = ds_train.with_options(options)
ds_val = ds_val.with_options(options)
ds_test = ds_test.with_options(options)

In [ ]:
utils.train_model(base_model, epochs, ds_train, train_batches, ds_val, None, False)

In [ ]:
print("Make folder...")
dir_name = datetime.datetime.now().strftime("%Y%m%d_%H%M%S") + "_Test"
os.mkdir(dir_name)

utils.saveModel(base_model, dir_name + "/base_model.h5")

In [ ]:
test_predict, test_labels, dir_name = ev.testModel(base_model, test_batches, dir_name)
ev.testModelWithThresholdChange(base_model, val_batches, test_predict, test_labels, dir_name)